# Lesson 5:Project Intro Varying The amount of noise

Augment the randomized response query from the previous project to allow for varying amount of randomness to be added.

eg:: I want you to be able to bias the first coin flip to be higher or lower probabilty of outputting a one
and then run the same experiment.

# Varying the amount of noise::
1)Add a new parameter to the query function.It will now accept the databse and some noise parameter which is a 	%.
The noise parameter should change is how likely for the first coin to be a one as opposed to zero.
2)Properly re-balance the results of query given this adjustable parameter.

In [1]:
import torch

# the number of entries in our database
num_entries = 5000

db = torch.rand(num_entries) > 0.5
db

tensor([0, 1, 0,  ..., 0, 1, 0], dtype=torch.uint8)

In [2]:
def get_parallel_db(db, remove_index):

    return torch.cat((db[0:remove_index], 
                      db[remove_index+1:]))

In [3]:
def get_parallel_dbs(db):

    parallel_dbs = list()

    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        parallel_dbs.append(pdb)
    
    return parallel_dbs

In [4]:
def create_db_and_parallels(num_entries):
    
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

# Project: Varying Amounts of Noise

In [5]:
def query(db, noise=0.2):
    
    true_result = torch.mean(db.float())

    first_coin_flip = (torch.rand(len(db)) < noise).float()
    second_coin_flip = (torch.rand(len(db)) < 0.5).float()

    augmented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip

    sk_result = augmented_database.float().mean()

    private_result = ((sk_result / noise) - 0.5) * noise / (1 - noise)

    return private_result, true_result

# Explanation of de-skewing of an augmented database

# Ques::
As per your formula::
P(true_db) = (P(augmented_db) – (1-noise)*0.5) / noise

If i co-relate it with given in lesson mentioned below::
private_result = ((sk_result / noise) - 0.5) * noise / (1 - noise)

private_result means true_db
sk_result   means augmented_db
Is it ok?

According to me these results of your formula P(true_db) and private_result are not matching, when i solve them.
Can you please help me?

# Ans::
@Nirupama Singh I took the formula for defining the first coin flip with varying amount of noise as given in the solution notebook here https://github.com/udacity/private-ai/blob/master/completed/Section%201%20-%20Differential%20Privacy.ipynb
After that I tries to explain how to then de-skew the result and the formula on which I arrived was same as that given in the notebook.
Indeed my formula is different from that in the lectures. This is because Andrew changed the way he was defining noise in between lecture and that is why the result is not same as the one in the lecture videos. To read more on this follow this thread: https://secureprivataischolar.slack.com/archives/CJCJJQ42W/p1562259776379800?thread_ts=1562254071.366600&cid=CJCJJQ42W (edited)
Henrique Mello
@Sumanyu Rosha This is the biggest problem in the videos I found so far. Your de-skewing is correct. The  lecture's equation is also correct if you change the definition of what noise is. First, Andrew uses noise as the probability of the coin landing heads. However, when he goes to the de-skewing part, he changes the definition out of the blue and now noise becomes the probability of the coin landing heads. To check that this is what is happening, consider this:

If H is the probability of landing heads, the probability of landing tails is (1 - H), right? So if the coin has 70% of probability of landing heads, it has 30% of landing tails. Now, this is the definitino he used first.

hence
`P(augmented_db) = P(true_db) * H + 0.5 * (1 - H)       Eq.1`
If you define H as noise, than the equation is the same you posted.

As I said before, Andrew changes the definition. He uses noise as the probability of the coin landing tails, T. That is, the equation to P(augmented_db) is now:
`P(augmented_db) = P(true_db)*(1 - T) + 0.5*T       Eq.2`
Here I made use of the fact that `prob of landing heads = H = (1 - T)` and `prob of landing tails = T = (1 - H)`.

Now if we take Eq.2 (which is the one Andrew used in the last equation you wrote) and say that `noise = T`,

`P(aug_db) = P(true_db) * (1 - noise) + 0.5*noise    Eq. 3`

From Eq.3 you can find the last equation you wrote.



